# Configurations

In [39]:
from sklearn import preprocessing
from IPython.display import display, HTML
import numpy as np
import pandas as pd
from sklearn.cross_validation import StratifiedKFold
from sklearn import cross_validation, metrics, grid_search, linear_model, svm
from sklearn.model_selection import GridSearchCV

Training_file = './Dataset/train_ZoGVYWq.csv'
Test_file     = './Dataset/test_66516Ee.csv'

Training_dtype = {'id':np.str, 'perc_premium_paid_by_cash_credit':np.float32, 
'age_in_days':np.float32, 
'Income':np.float32,
'Count_3-6_months_late':np.float32, 
'Count_6-12_months_late':np.float32,
'Count_more_than_12_months_late':np.float32, 
'application_underwriting_score':np.float32,
'no_of_premiums_paid':np.float32, 
'sourcing_channel':np.str, 
'residence_area_type':np.str,
'premium':np.int, 
'renewal':np.int}

Test_dtype = {'id':np.str, 'perc_premium_paid_by_cash_credit':np.float32, 
'age_in_days':np.float32, 
'Income':np.float32,
'Count_3-6_months_late':np.float32, 
'Count_6-12_months_late':np.float32,
'Count_more_than_12_months_late':np.float32, 
'application_underwriting_score':np.float32,
'no_of_premiums_paid':np.float32, 
'sourcing_channel':np.str, 
'residence_area_type':np.str,
'premium':np.int}


## 1) Loading Dataset - Train and test

In [18]:
# Loading dataset
trn_origin = pd.read_csv(Training_file, dtype=Training_dtype, na_values='')
tst_origin = pd.read_csv(Test_file, dtype=Test_dtype, na_values='')

display(trn_origin.head())
display(tst_origin.head())

#Check the shape of each dataset
print(trn_origin.shape)
print(tst_origin.shape)

,id,perc_premium_paid_by_cash_credit,age_in_days,Income,Count_3-6_months_late,Count_6-12_months_late,Count_more_than_12_months_late,application_underwriting_score,no_of_premiums_paid,sourcing_channel,residence_area_type,premium,renewal
0,110936,0.429,12058.0,355060.0,0.0,0.0,0.0,99.019997,13.0,C,Urban,3300,1
1,41492,0.010,21546.0,315150.0,0.0,0.0,0.0,99.889999,21.0,A,Urban,18000,1
2,31300,0.917,17531.0,84140.0,2.0,3.0,1.0,98.690002,7.0,C,Rural,3300,0
3,19415,0.049,15341.0,250510.0,0.0,0.0,0.0,99.570000,9.0,A,Urban,9600,1
4,99379,0.052,31400.0,198680.0,0.0,0.0,0.0,99.870003,12.0,B,Urban,9600,1


,id,perc_premium_paid_by_cash_credit,age_in_days,Income,Count_3-6_months_late,Count_6-12_months_late,Count_more_than_12_months_late,application_underwriting_score,no_of_premiums_paid,sourcing_channel,residence_area_type,premium
0,649,0.001,27384.0,51150.0,0.0,0.0,0.0,99.889999,7.0,A,Rural,3300
1,81136,0.124,23735.0,285140.0,0.0,0.0,0.0,98.930000,19.0,A,Urban,11700
2,70762,1.000,17170.0,186030.0,0.0,0.0,0.0,NaN,2.0,B,Urban,11700
3,53935,0.198,16068.0,123540.0,0.0,0.0,0.0,99.000000,11.0,B,Rural,5400
4,15476,0.041,10591.0,200020.0,1.0,0.0,0.0,99.169998,14.0,A,Rural,9600


(79853, 13)
(34224, 12)


## 1-2) Check the descriptive statistics for each dataset
- We can see that some columns have 'missing' values in it as counts of each column are different.

In [19]:
display(trn_origin.describe())
display(tst_origin.describe())

# age_in_days, Income, Count_3-6_months_late, Count_6-12_months_late	Count_more_than_12_months_late	application_underwriting_score	no_of_premiums_paid	premium

,perc_premium_paid_by_cash_credit,age_in_days,Income,Count_3-6_months_late,Count_6-12_months_late,Count_more_than_12_months_late,application_underwriting_score,no_of_premiums_paid,premium,renewal
count,79853.000000,79853.000000,7.985300e+04,79756.000000,79756.000000,79756.000000,76879.000000,79853.000000,79853.000000,79853.000000
mean,0.314291,18846.738281,2.088486e+05,0.248671,0.078188,0.060008,99.071289,10.863888,10924.507533,0.937410
std,0.334902,5208.866699,4.965778e+05,0.691676,0.436455,0.311902,0.739792,5.170848,9401.676542,0.242226
min,0.000000,7670.000000,2.403000e+04,0.000000,0.000000,0.000000,91.900002,2.000000,1200.000000,0.000000
25%,0.034000,14974.000000,1.080100e+05,0.000000,0.000000,0.000000,98.809998,7.000000,5400.000000,1.000000
50%,0.167000,18625.000000,1.665600e+05,0.000000,0.000000,0.000000,99.209999,10.000000,7500.000000,1.000000
75%,0.538000,22636.000000,2.520900e+05,0.000000,0.000000,0.000000,99.540001,14.000000,13800.000000,1.000000
max,1.000000,37602.000000,9.026260e+07,13.000000,17.000000,11.000000,99.889999,60.000000,60000.000000,1.000000


,perc_premium_paid_by_cash_credit,age_in_days,Income,Count_3-6_months_late,Count_6-12_months_late,Count_more_than_12_months_late,application_underwriting_score,no_of_premiums_paid,premium
count,34224.000000,34224.000000,3.422400e+04,34193.000000,34193.000000,34193.000000,32901.000000,34224.000000,34224.000000
mean,0.314454,18824.125000,2.028202e+05,0.238733,0.080718,0.058111,99.062012,10.890428,10834.721248
std,0.334058,5246.466797,2.702532e+05,0.686056,0.454576,0.307093,0.742933,5.216792,9263.074506
min,0.000000,7671.000000,2.403000e+04,0.000000,0.000000,0.000000,91.900002,2.000000,1200.000000
25%,0.034000,14972.000000,1.063975e+05,0.000000,0.000000,0.000000,98.800003,7.000000,5400.000000
50%,0.169000,18623.000000,1.650700e+05,0.000000,0.000000,0.000000,99.209999,10.000000,7500.000000
75%,0.540000,22636.000000,2.500200e+05,0.000000,0.000000,0.000000,99.529999,14.000000,13800.000000
max,1.000000,35785.000000,2.191455e+07,12.000000,10.000000,7.000000,99.889999,59.000000,60000.000000


## 2) Data preprocessing


### 2-0) Split X and Y dataset

In [20]:
trn_origin_X = trn_origin.loc[:,trn_origin.columns != 'renewal'].copy()
trn_origin_Y = trn_origin.loc[:,'renewal'].copy()

tst_origin_X = tst_origin.copy()

## 2-1) Imputating Missing values
- We can figure out columns containing missing values

In [21]:
# Check the missing values
display(trn_origin_X.isnull().sum())
display(tst_origin_X.isnull().sum())

id                                     0
perc_premium_paid_by_cash_credit       0
age_in_days                            0
Income                                 0
Count_3-6_months_late                 97
Count_6-12_months_late                97
Count_more_than_12_months_late        97
application_underwriting_score      2974
no_of_premiums_paid                    0
sourcing_channel                       0
residence_area_type                    0
premium                                0
dtype: int64

id                                     0
perc_premium_paid_by_cash_credit       0
age_in_days                            0
Income                                 0
Count_3-6_months_late                 31
Count_6-12_months_late                31
Count_more_than_12_months_late        31
application_underwriting_score      1323
no_of_premiums_paid                    0
sourcing_channel                       0
residence_area_type                    0
premium                                0
dtype: int64

### Imputation Strategy 1 : Mean

In [22]:
from sklearn.preprocessing import Imputer

# From .isnull().sum()
na_in_cols = ['Count_3-6_months_late', 'Count_6-12_months_late', 
            'Count_more_than_12_months_late', 'application_underwriting_score']

trn_imputed_X = trn_origin_X.copy()
tst_imputed_X = tst_origin_X.copy()

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(trn_imputed_X[na_in_cols])

trn_imputed_X[na_in_cols] = imp.transform(trn_imputed_X[na_in_cols])
tst_imputed_X[na_in_cols] = imp.transform(tst_imputed_X[na_in_cols])


### Imputation Strategy 2 : K-nearest Neighbors

In [ ]:
# ToDo

In [23]:
# Check the missing values AGAIN
display(trn_imputed_X.isnull().sum())
display(tst_imputed_X.isnull().sum())

id                                  0
perc_premium_paid_by_cash_credit    0
age_in_days                         0
Income                              0
Count_3-6_months_late               0
Count_6-12_months_late              0
Count_more_than_12_months_late      0
application_underwriting_score      0
no_of_premiums_paid                 0
sourcing_channel                    0
residence_area_type                 0
premium                             0
dtype: int64

id                                  0
perc_premium_paid_by_cash_credit    0
age_in_days                         0
Income                              0
Count_3-6_months_late               0
Count_6-12_months_late              0
Count_more_than_12_months_late      0
application_underwriting_score      0
no_of_premiums_paid                 0
sourcing_channel                    0
residence_area_type                 0
premium                             0
dtype: int64

## 2-2) Convert string values into dummy variables

In [24]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

cat_cols = ['sourcing_channel', 'residence_area_type']

# Concatenate trn and test X dataset
n_row_trn, ncol_trn = trn_imputed_X.shape
n_row_tst, ncol_tst = tst_imputed_X.shape

agg_data_X = pd.concat([trn_imputed_X, tst_imputed_X], axis=0)

for cat_col in cat_cols:
    print(' Convert string values from col : ', cat_col)

    just_dummy = pd.get_dummies(agg_data_X[cat_col], prefix=cat_col)
    #print(just_dummy)
    
    # Concatenate dummy columns into dataset
    agg_data_X = pd.concat([agg_data_X, just_dummy], axis=1)
    # Drop the origin column
    agg_data_X.drop(cat_col, axis=1, inplace=True)


# Split trn and test X dataset
trn_imputed_dum_X = agg_data_X.iloc[0:n_row_trn, :].copy()
tst_imputed_dum_X = agg_data_X.iloc[n_row_trn:, :].copy()

print(trn_imputed_dum_X.shape)
print(tst_imputed_dum_X.shape)

    #display(agg_data_X)
#     label_encoder = LabelEncoder()
#     onehot_encoder = OneHotEncoder(sparse=False)

#     trn_col_arr = trn_imputed[cat_col]
#     tst_col_arr = tst_imputed[cat_col]
    
#     ## Str to Integer encoding
#     label_encoder.fit(trn_col_arr)
#     trn_int_encoded = label_encoder.transform(trn_col_arr)
#     tst_int_encoded = label_encoder.transform(tst_col_arr)
#     #print(integer_encoded)

#     ## Integer to Binary encoding
#     trn_int_encoded = trn_int_encoded.reshape(len(trn_int_encoded), 1)
#     tst_int_encoded = tst_int_encoded.reshape(len(tst_int_encoded), 1)
#     onehot_encoder.fit(trn_int_encoded)
#     trn_onehot_encoded = onehot_encoder.transform(trn_int_encoded)
#     tst_onehot_encoded = onehot_encoder.transform(tst_int_encoded)
    
#     print(trn_onehot_encoded)
#     print(tst_onehot_encoded)
    
    ## Merge to the dataset
    

 Convert string values from col :  sourcing_channel
 Convert string values from col :  residence_area_type
(79853, 17)
(34224, 17)


## 2-3) Data standardization

In [36]:
cols_for_std = ['age_in_days', 'Income', 'Count_3-6_months_late', 
                'Count_6-12_months_late', 'Count_more_than_12_months_late', 'application_underwriting_score',
                'no_of_premiums_paid', 'premium']

scaler = preprocessing.StandardScaler().fit(trn_imputed_dum_X[cols_for_std])
trn_imputed_dum_X[cols_for_std] = scaler.transform(trn_imputed_dum_X[cols_for_std])
tst_imputed_dum_X[cols_for_std] = scaler.transform(tst_imputed_dum_X[cols_for_std])


## 3) Classification Models

In [52]:
n_fold = 10
kf = StratifiedKFold(trn_origin_Y, n_folds = n_fold)
scoring = {'AUC': 'roc_auc', 'F1': 'f1', 'Prec':'precision', 'Rec':'recall'}

## 3-1) Logistic Regression

### Hyperparameter optimization

In [57]:
# Parameters
lr_param = {'C':[0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.2, 0.5, 1, 5, 10],
           'tol' : [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 5, 10]}

lr_param = {'C':[0.001,  10],
           'tol' : [1e-7, 5, 10]}

lr_results = pd.DataFrame(np.empty((0,4), float), columns=['c', 'tol', 'AUROC', 'f1'])


def clf_LogisticLR(kf):
    clf_LR = linear_model.LogisticRegression()
    
    for trn_idx, val_idx in kf:
        clf = GridSearchCV(clf_LR, lr_param, n_jobs=8, scoring=scoring)
        clf.fit(trn_imputed_dum_X.loc[trn_idx,:], trn_origin_Y.loc[trn_idx], refit='roc_auc')



## 3-x) DO PREDICTION

In [56]:
clf_LogisticLR(kf)

ValueError: For multi-metric scoring, the parameter refit must be set to a scorer key to refit an estimator with the best parameter setting on the whole data and make the best_* attributes available for that metric. If this is not needed, refit should be set to False explicitly. True was passed.